In [ ]:
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter

In [ ]:
chat_df = pd.read_csv('chat_data/chat_history 2024-11-08 v3.csv')
members_df = pd.read_csv('chat_data/chat_members 2024-11-08 v3.csv')
reacts_df = pd.read_csv('chat_data/reactions 2024-11-08.csv')
members_reacts_df = pd.read_csv('chat_data/members_reactions 2024-11-08.csv')
chat_df

In [ ]:
chat_df.dtypes

In [ ]:
chat_df.text = chat_df.text.fillna("")
chat_df.edited = pd.to_datetime(chat_df.edited, errors='coerce')
chat_df.reply_to_id = chat_df.reply_to_id.fillna(0).astype(int)
chat_df.date = pd.to_datetime(chat_df.date, errors='coerce')
chat_df.dtypes

## Анализ сообщений

### Количество сообщений

In [ ]:
# Количество сообщений, отправленных участниками чата
message_count = chat_df.groupby('from_id')[['id']].count().merge(members_df, left_on='from_id', right_on='user_id')[['username', 'id']]\
                       .sort_values('id', ascending=False).reset_index(drop=True)
message_count[:20]

In [ ]:
# Круговая диаграмма доли сообщений пользователей
main_members = message_count[message_count.id / message_count.id.sum() > 0.05]
other_members = pd.Series({'username': 'Остальные', 
                           'id': message_count[message_count.id / message_count.id.sum() <= 0.05].id.sum()})
(pd.concat([main_members, other_members.to_frame().T], ignore_index=True)
 .set_index('username')
 .plot.pie(y='id', figsize=(12, 10)))

In [ ]:
# Топ 100 самый часто встречающихся сообщений
unique_messeges = chat_df[['id', 'text']]
unique_messeges['text'] = unique_messeges.text.apply(lambda x: x.lower() if x.count(' ') != 0 else None).dropna()
unique_messeges = (unique_messeges.groupby('text', as_index=False)['id'].count()
                   .sort_values('id', ascending=False)
                   .reset_index(drop=True))
for i, row in unique_messeges[unique_messeges.id > 1][:100].iterrows():
    print(row['text'], row['id'], sep=' | ', end='\n\n')

### Ответы

In [ ]:
# Вывести самые частые пары сообщений:
# оригинальное сообщение и прямой ответ на него;
# пара идущих друг за другом сообщений (что не всегда является
# хорошим примером для будущего обучающего датасета)
pairs = (chat_df.loc[1:, ['id', 'from_id', 'text', 'reply_to_id']]
         .merge(members_df, left_on='from_id', right_on='user_id')
         .reset_index(drop=True)
         .rename(columns={'id': 'id2', 'from_id': 'from_id2', 'text': 'text2', 'username': 'user2'})
         [['id2', 'user2', 'text2', 'reply_to_id']])
pairs['reply_to_id'] = pd.Series([
    prev_id if repi == 0 else repi
    for prev_id, repi in zip(chat_df.id[:-1].to_list(), 
                             pairs.reply_to_id.to_list())])
pairs = pairs.merge(chat_df[['id', 'from_id', 'text']]
                    .merge(members_df, left_on='from_id', right_on='user_id')
                    .rename(columns={'username': 'user'})
                    [['id', 'user', 'text']], 
                    left_on='reply_to_id', right_on='id')
pairs = pairs[['id', 'user', 'text', 'id2', 'user2', 'text2']]
pairs['text'] = pairs.text.str.lower()
pairs['text2'] = pairs.text2.str.lower()


def print_most_friquent_pairs(df):
    print(*list(sorted(
        filter(lambda x: x[1] > 1,
               Counter(list(zip(df.text.to_list(),
                                df.text2.to_list()))).items()),
        key=lambda x: x[1], reverse=True)),
          sep='\n\n')

# pairs
print_most_friquent_pairs(pairs)

In [ ]:
# Кому чаще всего отвечали сообщением?
def print_most_replied_users(chat_df, users_df):
    return (chat_df.merge(chat_df[['id', 'from_id']].rename(
        columns={'id': 'repl_id', 'from_id': 'pers_id'}), 
                          left_on='reply_to_id', 
                          right_on='repl_id', 
                          how='inner')
                   .groupby('pers_id')[['repl_id']].count()
                   .merge(users_df, left_on='pers_id', right_on='user_id')
                   [['username', 'repl_id']]
                   .sort_values('repl_id', ascending=False)
                   .reset_index(drop=True)
          )

print_most_replied_users(chat_df, members_df)[:20]

In [ ]:
# Горизонтальная столбчатая диаграмма для данных выше
(chat_df.merge(chat_df[['id', 'from_id']].rename(
    columns={'id': 'repl_id', 'from_id': 'pers_id'}),
              left_on='reply_to_id', right_on='repl_id', how='inner')
        .groupby('pers_id')[['repl_id']].count()
        .merge(members_df, left_on='pers_id', right_on='user_id')
        [['username', 'repl_id']]
        .sort_values('repl_id', ascending=False)
        .reset_index(drop=True)[19::-1]
        .plot.barh(x='username', figsize=(14, 6), title='Количество ответов на сообщения участников чата'));

In [ ]:
# Кто чаще всего отвечает на сообщения?
def print_most_replying_users(chat_df, users_df):
    return (chat_df[chat_df.reply_to_id != 0].groupby('from_id')[['id']].count()
          .merge(users_df, left_on='from_id', right_on='user_id', how='left')[['username', 'id']]
          .sort_values('id', ascending=False).reset_index(drop=True)
          )

print_most_replying_users(chat_df, members_df)[:20]

In [ ]:
# Горизонтальная столбчатая диаграмма для данных выше
(chat_df[chat_df.reply_to_id != 0].groupby('from_id')[['id']].count()
        .merge(members_df, left_on='from_id', right_on='user_id')[['username', 'id']]
        .sort_values('id', ascending=False).reset_index(drop=True)[19::-1]
        .plot.barh(x='username', figsize=(14, 6), title='Количество сообщений, на которые ответили участники чата'));

In [ ]:
# Кто кому чаще всего отвечает
repl = sorted(list(Counter(zip(pairs.user, pairs.user2)).items()), key=lambda x: x[1], reverse=True)
repl = [f'{u2} -> {u1} ({n})' for (u1, u2), n in repl if u1 != u2]
print(*repl[:20], sep='\n\n')

In [ ]:
# На какие сообщения кто отвечает
repl = sorted(list(Counter(zip(pairs.text, pairs.user2)).items()), key=lambda x: x[1], reverse=True)
repl = [f'{u2} -> {txt} ({n})' for (txt, u2), n in repl]
print(*repl[:20], sep='\n\n')

In [ ]:
# Какими сообщениями кому отвечают
repl = sorted(list(Counter(zip(pairs.user, pairs.text2)).items()), key=lambda x: x[1], reverse=True)
repl = [f'{txt2} -> {u1} ({n})' for (u1, txt2), n in repl]
print(*repl[:20], sep='\n\n')

In [ ]:
# Кто кому и что чаще всего пишет
repl = sorted(list(Counter(zip(pairs.user, pairs.user2, pairs.text2)).items()), key=lambda x: x[1], reverse=True)
repl = [f'{u2}: {txt} -> {u1} ({n})' for (u1, u2, txt), n in repl if u1 != u2 and n > 1]
print(*repl[:20], sep='\n\n')

In [ ]:
# Кто кому и на что чаще всего отвечает
repl = sorted(list(Counter(zip(pairs.user, pairs.user2, pairs.text)).items()), key=lambda x: x[1], reverse=True)
repl = [f'{u2} -> {u1}: {txt} ({n})' for (u1, u2, txt), n in repl if u1 != u2 and n > 1]
print(*repl[:20], sep='\n\n')

In [ ]:
# Кто кому что отвечает на какое сообщение чаще всего
repl = sorted(list(Counter(zip(pairs.user, pairs.user2, pairs.text, pairs.text2)).items()), key=lambda x: x[1], reverse=True)
repl = [f'{u2}: {txt2} -> {u1}: {txt1} ({n})' for (u1, u2, txt1, txt2), n in repl if u1 != u2 and n > 1]
print(*repl[:20], sep='\n\n')

### Отредактированные сообщения

In [ ]:
# Кто редактировал больше всего своих сообщений?
edited_chat = chat_df[chat_df.edited != 0].groupby('from_id')[['edited']].count()\
                     .merge(members_df, left_on='from_id', right_on='user_id')[['username', 'edited']]\
                     .sort_values('edited', ascending=False).reset_index(drop=True)
edited_chat[:20]

In [ ]:
ratio_edit = message_count[message_count.id >= 250].merge(edited_chat, on='username', how='left')
ratio_edit.edited  = ratio_edit.edited / ratio_edit.id
ratio_edit = ratio_edit.sort_values('edited', ascending=False).reset_index(drop=True)
ratio_edit

In [ ]:
# Сколько времени прошло до последнего изменения сообщения
edited = chat_df[~chat_df.edited.isna()]
edited.edited  = edited.edited - edited.date
(edited.groupby(edited[edited.edited.dt.seconds <= 60].edited.dt.floor('1s')).count()[['id']]
       .rename(columns={'id': 'id_count'})
       .plot.area(figsize=(14, 6), title='Количество сообщений по времени с момента последнего изменения'))

In [ ]:
uid = ''  # идентификатор пользователя для индивидуального графика его времени последнего измения сообщений
(edited[edited.from_id == uid].groupby(edited[edited.edited.dt.seconds <= 60].edited.dt.floor('1s')).count()[['id']]
       .rename(columns={'id': 'id_count'})
       .plot.area(figsize=(14, 6), title='Количество сообщений по времени с момента последнего изменения\n'
                  f'{members_df.loc[members_df.user_id == uid, ["username"]].values[0][0]}'));

In [ ]:
# Сколько для каждого суммарно прошло времени от написания до последнего редактирования всех сообщений пользователя
(edited.groupby('from_id', as_index=False).edited.sum()\
      .merge(members_df, how='left' , left_on='from_id', right_on='user_id')[['username', 'edited']]
      .sort_values('edited').values.tolist())

### Время отправки сообщений

In [ ]:
# Графики количества отправленных сообщений
# chat_df.date = pd.to_datetime(chat_df.date, unit='s') + datetime.timedelta(hours=10)

chat_df.groupby(chat_df.date.dt.to_period('D')).count()[['id']]\
       .rename(columns={'id': 'id_count'}).plot.area(figsize=(14, 6), title='Количество сообщений по дням')

In [ ]:
uid = ''  # # идентификатор пользователя для индивидуального графика количества сообщений по дням
(chat_df[chat_df.from_id == uid].groupby(chat_df.date.dt.to_period('D')).count()[['id']]
        .rename(columns={'id': 'id_count'})
        .plot.area(figsize=(14, 6), title=f"Количество сообщений по дням" +
                   f"({members_df.loc[members_df.user_id == uid, ['username']].values[0][0]})"));

In [ ]:
# Топ 10 самых активных дня
(chat_df.groupby(chat_df.date.dt.to_period('D')).count()[['id']]
        .rename(columns={'id': 'id_count'}).sort_values('id_count', ascending=False)[:10]);

In [ ]:
# График количества сообщений по неделям
(chat_df.groupby(chat_df.date.dt.to_period('W')).count()[['id']]
        .rename(columns={'id': 'id_count'})
        .plot.area(figsize=(14, 6), title='Количество сообщений за неделю'))
plt.xticks(rotation=90);

In [ ]:
# Количество сообщений по дням недели
week = chat_df.groupby(chat_df.date.dt.to_period('D').dt.weekday).count()[['id']].rename(columns={'id': 'id_count'})
week.index = ['Понедельник', 'вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']
week.plot.bar(figsize=(14, 6), title='Количество сообщений по дням недели')
plt.xticks(rotation=0);

In [ ]:
# Так как чат активен вплоть до 3 часов ночи, почему бы не сдвинуть день на 4 часа, 
# тем самым прибавив активности тем дням, которые ещё не закончились (то есть активные участники ещё не пошли спать)
chat_df.date = chat_df.date - datetime.timedelta(hours=4)

act_week = chat_df.groupby(chat_df.date.dt.to_period('D').dt.weekday).count()[['id']].rename(columns={'id': 'id_count'})
act_week.index = ['Понедельник', 'вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']
act_week.plot.bar(figsize=(14, 6), title='Количество сообщений во времена наибольшей активности по дням недели')
plt.xticks(rotation=0)

chat_df.date = chat_df.date + datetime.timedelta(hours=4)

In [ ]:
(chat_df.groupby(chat_df['date'].dt.to_period('H').dt.hour).count()[['id']]
        .rename(columns={'id': 'id_count'})
        .plot.bar(figsize=(14, 6), title='Количество сообщений в день по часам'))

In [ ]:
# Количество активных участников по часам
unique_users = chat_df.groupby(chat_df['date'].dt.to_period('H')).agg({'from_id': 'nunique'})
unique_users.index =  unique_users.index.hour
unique_users.groupby(level=0).mean().plot.bar(figsize=(14, 6), title='Среднее количество уникальных пользователей по часам')

In [ ]:
# Кто больше всего активен в конкретный час
mess_per_hour = chat_df.groupby([chat_df['date'].dt.to_period('H').dt.hour, 'from_id']).agg({'id': 'count'}).reset_index()
mess_per_hour.groupby('date')[['id']].idxmax().merge(mess_per_hour, left_on='id', right_index=True)[['from_id', 'id_y']]\
             .merge(members_df, left_on='from_id', right_on='user_id', how='left')[['username', 'id_y']].plot.bar(figsize=(14, 6))
plt.xticks(rotation=0)
plt.show()

mess_per_hour.groupby('date')[['id']].idxmax().merge(mess_per_hour, left_on='id', right_index=True)[['from_id', 'id_y']]\
.merge(members_df, left_on='from_id', right_on='user_id', how='left')[['username', 'id_y']]

### Стикеры

In [ ]:
stickers = chat_df[chat_df.text.str.startswith('File: <chat_data') & chat_df.text.str.endswith('>')][['from_id', 'text']]\
                  .merge(members_df, left_on='from_id', right_on='user_id')[['username', 'text']]\
                  .rename(columns={'text': 'sticker'})
stickers

In [ ]:
# Самые используемые стикеры
stickers.groupby('sticker')[['username']].count().rename(columns={'username': 'count'})\
        .sort_values('count', ascending=False).reset_index()[:20]

In [ ]:
import cv2
import os
from PIL import Image


fig, axs = plt.subplots(4, 5, figsize=(15, 12))
fig.subplots_adjust(wspace=0.1, hspace=0.4)
i = 0
for s in (stickers.groupby('sticker')[['username']].count().rename(columns={'username': 'count'})\
          .sort_values('count', ascending=False).reset_index().iterrows()):
    sn = s[1]['sticker']
    file = sn[sn.index('<') + 1: sn.index('>')]
    if not os.path.exists('.'.join(file.split('.')[:-1]) + '.jpg'):
        if file.split('.')[-1] == 'webp':
            image_webp = Image.open(file)
            
            print(file, end='\t||\t')
            file = '.'.join(file.split('.')[:-1]) + '.jpg'
            print(file)
            
            image_webp.convert('RGB').save(file)
            
        elif file.split('.')[-1] == 'webm':
            video_capture = cv2.VideoCapture(file)
            success, image = video_capture.read()

            print(file, end='\t||\t')
            file = '.'.join(file.split('.')[:-1]) + '.jpg'
            print(file)
            
            cv2.imwrite(file, image)
            
        elif file.split('.')[-1] == 'tgs' and os.path.exists('.'.join(file.split('.')[:-1]) + '.webp'):
            # convertLottie2Webp(file, '.'.join(file.split('.')[:-1]) + '.webp')
            # lottie_convert.py "ChatExport_2024-02-15/stickers/AnimatedSticker (53).tgs" "ChatExport_2024-02-15/stickers/AnimatedSticker (53).webp"
            file = '.'.join(file.split('.')[:-1]) + '.webp'
            image_webp = Image.open(file)
            
            print(file, end='\t||\t')
            file = '.'.join(file.split('.')[:-1]) + '.jpg'
            print(file)
            
            image_webp.convert('RGB').save(file)
    else:
        file = '.'.join(file.split('.')[:-1]) + '.jpg'
    
    image = cv2.imread(file)
    if image is None:
        continue

    # print(file, image)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    axs[i // 5, i % 5].imshow(image_rgb)
    axs[i // 5, i % 5].set_title(f"{i + 1} ({s[1]['count']})")
    axs[i // 5, i % 5].axis('off')
    
    i += 1
    if i > 19:
        break

plt.show()

In [ ]:
# Любимые стикеры у участников чата
members_stickers = stickers[:]
members_stickers['count'] = pd.Series([0] * stickers.shape[0])
members_stickers = (members_stickers.groupby(['username', 'sticker'], as_index=False).count()
                                    .sort_values(['username', 'count', 'sticker'], ascending=[True, False, True]))
members_stickers = (members_stickers.groupby('username', as_index=False).agg({'count': 'max', 'sticker': 'first'})
                                    .sort_values('count', ascending=False)
                                    .reset_index(drop=True))
members_stickers.head(10)

In [ ]:
import cv2
import os
from PIL import Image


nrows = members_stickers.username.count() // 5 + 1
fig, axs = plt.subplots(nrows, 5, figsize=(12, 2.4 * nrows))
fig.subplots_adjust(wspace=0.2, hspace=0.3)
i = 0
for s in (members_stickers.iterrows()):
    sn = s[1]['sticker']
    file = sn[sn.index('<') + 1: sn.index('>')]
    if not os.path.exists('.'.join(file.split('.')[:-1]) + '.jpg'):
        if file.split('.')[-1] == 'webp':
            image_webp = Image.open(file)
            
            print(file, end='\t||\t')
            file = '.'.join(file.split('.')[:-1]) + '.jpg'
            print(file)
            
            image_webp.convert('RGB').save(file)
            
        elif file.split('.')[-1] == 'webm':
            video_capture = cv2.VideoCapture(file)
            success, image = video_capture.read()

            print(file, end='\t||\t')
            file = '.'.join(file.split('.')[:-1]) + '.jpg'
            print(file)
            
            cv2.imwrite(file, image)
            
        elif file.split('.')[-1] == 'tgs' and os.path.exists('.'.join(file.split('.')[:-1]) + '.webp'):
            # convertLottie2Webp(file, '.'.join(file.split('.')[:-1]) + '.webp')
            # lottie_convert.py "ChatExport_2024-02-15/stickers/AnimatedSticker (53).tgs" "ChatExport_2024-02-15/stickers/AnimatedSticker (53).webp"
            file = '.'.join(file.split('.')[:-1]) + '.webp'
            image_webp = Image.open(file)
            
            print(file, end='\t||\t')
            file = '.'.join(file.split('.')[:-1]) + '.jpg'
            print(file)
            
            image_webp.convert('RGB').save(file)
    else:
        file = '.'.join(file.split('.')[:-1]) + '.jpg'
    
    image = cv2.imread(file)
    if image is None:
        continue

    # print(file, image)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    axs[i // 5, i % 5].imshow(image_rgb)
    axs[i // 5, i % 5].set_title(f"{s[1].username} ({s[1]['count']})")
    axs[i // 5, i % 5].axis('off')
    
    i += 1

if i <= nrows*5:
    while i < nrows*5:
        axs[i // 5, i % 5].axis('off')
        i += 1

plt.show()

In [ ]:
# Вывод топ 10 стикеров и конвертация в jpg
import cv2
import os
from PIL import Image


fig, axs = plt.subplots(2, 5, figsize=(15, 6))
fig.subplots_adjust(wspace=0.1, hspace=0.4)
i = 0
for s in (stickers[stickers.username == 'Cthulhu Fxhnsky']
                  .groupby('sticker')[['username']].count()
                  .rename(columns={'username': 'count'})
                  .sort_values('count', ascending=False)
                  .reset_index().iterrows()):
    sn = s[1]['sticker']
    file = sn[sn.index('<') + 1: sn.index('>')]
    if not os.path.exists('.'.join(file.split('.')[:-1]) + '.jpg'):
        if file.split('.')[-1] == 'webp':
            image_webp = Image.open(file)
            
            print(file, end='\t||\t')
            file = '.'.join(file.split('.')[:-1]) + '.jpg'
            print(file)
            
            image_webp.convert('RGB').save(file)
            
        elif file.split('.')[-1] == 'webm':
            video_capture = cv2.VideoCapture(file)
            success, image = video_capture.read()

            print(file, end='\t||\t')
            file = '.'.join(file.split('.')[:-1]) + '.jpg'
            print(file)
            
            cv2.imwrite(file, image)
            
        elif file.split('.')[-1] == 'tgs' and os.path.exists('.'.join(file.split('.')[:-1]) + '.webp'):
            # convertLottie2Webp(file, '.'.join(file.split('.')[:-1]) + '.webp')
            # lottie_convert.py "ChatExport_2024-02-15/stickers/AnimatedSticker (53).tgs" "ChatExport_2024-02-15/stickers/AnimatedSticker (53).webp"
            file = '.'.join(file.split('.')[:-1]) + '.webp'
            image_webp = Image.open(file)
            
            print(file, end='\t||\t')
            file = '.'.join(file.split('.')[:-1]) + '.jpg'
            print(file)
            
            image_webp.convert('RGB').save(file)
    else:
        file = '.'.join(file.split('.')[:-1]) + '.jpg'
    
    image = cv2.imread(file)
    if image is None:
        continue

    # print(file, image)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    axs[i // 5, i % 5].imshow(image_rgb)
    axs[i // 5, i % 5].set_title(f"{i + 1} ({s[1]['count']})")
    axs[i // 5, i % 5].axis('off')
    
    i += 1
    if i > 9:
        break

plt.show()

## Анализ слов

In [ ]:
# Вывод топ 20 слов по частоте
import re


all_words = []
for m in chat_df[~chat_df.text.str.startswith('File: <')].text.str.split().to_list():
    all_words.extend(m)

all_words = [re.sub(r'[^0-9a-zA-Zа-яёА-ЯЁ\s]', '', w) for w in all_words]
print(*sorted(Counter(all_words).items(), key=lambda x: x[1], reverse=True)[:20], sep='\n')

### Токенизация

In [ ]:
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


nltk.data.path.append("bigdata/local_nltk")


def collect_lists(*args):
    all_els = []
    for l in args:
        # print(l.to_list(), end='\n' + '_' * 20 + '\n')
        arg_list = []
        for el in l.to_list():
            arg_list.extend(el)
        
        all_els.extend(arg_list)
    
    return all_els


def to_words(message):
    # print(message)
    # return message
    
    tokens = []
    for w in message:
        # if 'http://' in w or 'https://' in w:
        #     continue
        
        tokens.extend(word_tokenize(w, language='russian'))
    
    lowercase_tokens = [m.lower() for m in tokens]
    
    stop_words = set(stopwords.words('russian'))
    filtered_tokens = [w for w in lowercase_tokens if w not in stop_words]
    
    return list(filter(lambda x: x, filtered_tokens))
    # print(*sorted(Counter(tokenized_words).items(), 
    #               key=lambda x: x[1], reverse=True)[1:21], sep='\n')


# Считаем количество слов для каждого участника
words_df = chat_df[~chat_df.text.str.startswith('File: <')][['from_id', 'text']]
words_df.text = words_df.text.str.split().to_list()
words_df = words_df.groupby('from_id').agg({'text': collect_lists}).reset_index()
words_df.text = words_df.text.apply(to_words)
words_df[:20]

In [ ]:
# Количество слов каждого участника чата
words_count = words_df[:]
words_count.text = words_count.text.apply(len)
words_count = words_count.merge(members_df, left_on='from_id', right_on='user_id')[['username', 'text']]\
                         .sort_values('text', ascending=False).reset_index(drop=True)
words_count[:20]

In [ ]:
# Токены, используемые каждым участником чата, отсортированные по частоте
def top10words(wl):
    wlt = sorted(list(Counter(wl).items()), key=lambda x: x[1], reverse=True)
    return wlt

word_stat = words_df[:]
word_stat.text = words_df.text.apply(top10words)
word_stat = word_stat.merge(members_df, left_on='from_id', right_on='user_id', how='left')[['username', 'text']].reset_index(drop=True)

for u, wl in sorted(zip(word_stat.username, word_stat.text), key=lambda x: len(x[1]), reverse=True)[:20]:
    wl = [(w, k) for w, k in wl if k > 1]
    if len(wl) > 1:
        print(u, end='\n\n')
        print(wl, end='\n\n\n')

In [ ]:
# Словарный запас каждого участника чата
unique_words = word_stat[:]
unique_words['uw'] = unique_words.text.apply(len)
unique_words.sort_values('uw', ascending=False).reset_index(drop=True).loc[:19, ['username', 'uw']]

In [ ]:
uwc = words_count.merge(unique_words[['username', 'uw']], on='username')
uwc['uwp'] = uwc.uw / uwc.text
uwc[uwc.text > 500].sort_values('uwp', ascending=False)

In [ ]:
# Кто сколько использует конкретные слова
words_users_df = words_df.explode('text')
words_users_df = words_users_df.groupby('text')['from_id'].apply(list).reset_index()
words_users_df.from_id = words_users_df.from_id.apply(
    lambda x: sorted(list(Counter(x).items()), key=lambda x: x[1], reverse=True))

ud = {ui: nm for ui, nm in zip(members_df.user_id, members_df.username)}
words_users_df.from_id = words_users_df.from_id.apply(lambda x: [(ud[u], k) for u, k in x])
words_users_df['word_count'] = words_users_df.from_id.apply(lambda x: sum(k for _, k in x))
words_users_df = words_users_df.sort_values('word_count', ascending=False).reset_index(drop=True)
words_users_df[words_users_df.word_count > 1]

In [ ]:
# Лемматизируем текста сообщений
import pymorphy2


lemmatizer = pymorphy2.MorphAnalyzer()
lemm_words_df = words_df[:]
lemm_words_df.text = lemm_words_df.text.apply(lambda x: [lemmatizer.parse(word)[0].normal_form for word in x])
lemm_words_df[:20]

In [ ]:
# Кто сколько использует конкретные (лемматизированные) слова
lemm_words_users_df = lemm_words_df.explode('text')
lemm_words_users_df = lemm_words_users_df.groupby('text')['from_id'].apply(list).reset_index()
lemm_words_users_df.from_id = lemm_words_users_df.from_id.apply(
    lambda x: sorted(list(Counter(x).items()), key=lambda x: x[1], reverse=True))

ud = {ui: nm for ui, nm in zip(members_df.user_id, members_df.username)}
lemm_words_users_df.from_id = lemm_words_users_df.from_id.apply(lambda x: [(ud[u], k) for u, k in x])
lemm_words_users_df['word_count'] = lemm_words_users_df.from_id.apply(lambda x: sum(k for u, k in x))
lemm_words_users_df = lemm_words_users_df.sort_values('word_count', ascending=False).reset_index(drop=True)
lemm_words_users_df[lemm_words_users_df.word_count > 1]

In [ ]:
# Слово, и кто сколько его использовал в своих сообщениях
for i, line in lemm_words_users_df[lemm_words_users_df.text.str.startswith('your_word_here')].iterrows():
    print(line.text, end='\n\n')
    print(*line.from_id[:10 if len(line.from_id) >= 10 else len(line.from_id)], sep='\n', end='\n\n\n')

In [ ]:
# Кто использовал слова, встречающиеся в чате всего 1 раз
one_word_df = lemm_words_users_df[lemm_words_users_df.word_count == 1][['text', 'from_id']]
one_word_df.loc[:, 'from_id'] = one_word_df.from_id.apply(lambda x: x[0][0])
one_word_df = one_word_df.groupby('from_id').agg({'text': list}).reset_index()
one_word_df = one_word_df.loc[one_word_df['text'].apply(len).sort_values(ascending=False).index].reset_index(drop=True)
for u, w in zip(one_word_df.from_id, one_word_df.text):
    if len(w) > 100:
        print(u, w[:100], sep='    ', end='\n\n')
        continue

    print(u, w, sep='    ', end='\n\n')

### Пинги

In [ ]:
# Анализ пингов в чате
pins_df = chat_df[['from_id', 'text']]


def get_pins(text):
    if re.match(r'@[\w\d]+', text):
        return re.findall(r'@[\w\d]+', text)

    return ''


pins_df.loc[:, 'text'] = pins_df.text.apply(get_pins)
pins_df = (pins_df[pins_df.text != '']
                  .rename(columns={'text': 'pins'})
                  .merge(members_df, how='left', left_on='from_id', right_on='user_id')
                  [['username', 'pins']]
                  .reset_index(drop=True))
pins_df

In [ ]:
# Какие пинги встречаются чаще всего?
pin_list = []
for pin in pins_df.pins:
    pin_list.extend(pin)

print(*sorted(Counter(pin_list).items(), key=lambda x: x[1], reverse=True)[:20], sep='\n')

In [ ]:
# Кто чаще всего кого-либо пингует*
def collect_lists(*args):
    all_els = []
    for l in args:
        # print(l.to_list(), end='\n' + '_' * 20 + '\n')
        arg_list = []
        for el in l.to_list():
            arg_list.extend(el)
        
        all_els.extend(arg_list)
    
    return all_els


pin_count = pins_df.groupby('username', as_index=False).agg({'pins': collect_lists})
pin_count.pins = pin_count.pins.apply(len)
pin_count = pin_count.sort_values('pins', ascending=False).reset_index(drop=True)
pin_count[['username', 'pins']]
# pin_count.pins = max_pin_df.pins.apply(len)
# pin_count[['username', 'pins']]

In [ ]:
# Кто кого пингует
def all_pins(pins):
    pinl = sorted(list(Counter(pins).items()), key=lambda x: x[1], reverse=True)
    # maxp = max([k1 for _, k1 in pinl])
    return [(p, k) for p, k in pinl]


member_pins = pins_df.groupby('username', as_index=False).agg({'pins': collect_lists})
member_pins.pins = member_pins.pins.apply(all_pins)
member_pins = member_pins[['username', 'pins']]
for u, p in sorted(zip(member_pins.username, member_pins.pins), 
                   key=lambda x: sum(x[1][i][1] for i in range(len(x[1]))), reverse=True):
    print(u, end='\n\n')
    print(*p, sep='\n', end='\n\n\n')

In [ ]:
# Какие пинги кто использовал
pin_members = pd.DataFrame({}, columns=['pin', 'users'])

for i in range(pins_df.shape[0]):
    for j in range(len(pins_df.loc[i].pins)):
        pin_members = pd.concat([pin_members, pd.DataFrame({
            'pin': [pins_df.loc[i].pins[j]], 'users': [pins_df.loc[i].username]
            })], 
            ignore_index=True)

pin_members = pin_members.groupby('pin', as_index=False).users.apply(all_pins)
for p, u in sorted(zip(pin_members.pin, pin_members.users), key=lambda x: sum(x[1][i][1] for i in range(len(x[1]))), reverse=True):
    print(p, end='\n\n')
    print(*u, sep='\n', end='\n\n\n')
    

### Эмодзи

In [ ]:
import emoji
import re


def collect_lists(*args):
    all_els = []
    for l in args:
        # print(l.to_list(), end='\n' + '_' * 20 + '\n')
        arg_list = []
        for el in l:
            arg_list.extend(el)
        
        all_els.extend(arg_list)
    
    return all_els


# Анализ эмодзи
emoji_df = words_df[:]
emoji_df.text = emoji_df.text.apply(lambda x: [
    emoji.emoji_list(w) for w in filter(lambda y: re.match(r'[^а-яёА-ЯЁa-zA-Z.,\d]', y), x) 
    if emoji.emoji_count(w) > 0])
emoji_df.text = (emoji_df.text
                        .apply(lambda x: [[em['emoji'] for em in el] for el in x])
                        .apply(collect_lists))

emoji_count_df = emoji_df[:]
emoji_count_df.text = emoji_df.text.apply(lambda el: sorted(
    list(Counter(el).items()), key=lambda x: x[1], reverse=True))
emoji_count_df = (emoji_count_df.merge(members_df, left_on='from_id', right_on='user_id')
                  [['username', 'text']]
                  .rename(columns={'text': 'emojies'}))

# Кто какие эмодзи отправил
for u, el in sorted(zip(emoji_count_df.username, emoji_count_df.emojies), 
                    key=lambda x: sum(k for _, k in x[1]), reverse=True):
    if len(el) > 0:
        print(u, el, sep='    ', end='\n\n')
    

In [ ]:
emoji_users_df = emoji_df.explode('text')
emoji_users_df = emoji_users_df.groupby('text')['from_id'].apply(list).reset_index()
emoji_users_df.from_id = emoji_users_df.from_id.apply(lambda x: sorted(
    list(Counter(x).items()), key=lambda x: x[1], reverse=True))

ud = {ui: nm for ui, nm in zip(members_df.user_id, members_df.username)}
# print(ud)
emoji_users_df.from_id = emoji_users_df.from_id.apply(lambda x: [(ud[ui], k) for ui, k in x])

# Какие эмодзи кем были отправлены
for em, ul in sorted(zip(emoji_users_df.text, emoji_users_df.from_id), 
                     key=lambda x: sum(k for _, k in x[1]), reverse=True):
    print(em, ul, sep='    ', end='\n\n')

In [ ]:
# Какая доля эмодзи среди всех слов
emoji_stat = emoji_df[:]
emoji_stat.text = emoji_stat.text.apply(len)
emoji_stat = (emoji_stat[emoji_stat.text > 0]
                        .sort_values('text', ascending=False)
                        .reset_index(drop=True)
                        .merge(members_df[
                            members_df.user_id.str.startswith('user') | members_df.user_id.str.startswith('channel')
                            ], 
                               how='left', left_on='from_id', right_on='user_id')
                        [['username', 'text']]
                        .rename(columns={'text': 'emojies'}))
emoji_stat = emoji_stat.merge(words_count[words_count.text > 300], on='username')
emoji_stat.text = emoji_stat.emojies / emoji_stat.text
emoji_stat = emoji_stat.sort_values('emojies', ascending=False).reset_index(drop=True)
emoji_stat

In [ ]:
# Кто сколько использовал эмодзи в сообщениях
emoji_stat.sort_values('text', ascending=False).reset_index(drop=True)

In [ ]:
words_df.text.apply(lambda x: list(filter(lambda y: re.match(r'[^а-яёА-ЯЁa-zA-Z.,\d]', y), x)) or None).dropna()

### Визуализация слов

In [ ]:
import re

import nltk.data
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.data.path.append("bigdata/local_nltk")


def get_tokens(df):
    """Токенизация текста по словам, очистка от стоп слов и прочих символов"""
    # cleaned_text = [re.sub(r'[^a-zA-Zа-яёА-ЯЁ@#\s]', '', m) 
    #                 for m in chat_df[~chat_df.text.str.startswith('File: <')].text]

    tokens = []
    for m in df.text:
        # if 'http://' in m or 'https://' in m:
        #     pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        #     # links = re.findall(pattern, m)
        #     m = re.sub(pattern, '', m)
        
        tokens.append(word_tokenize(m, language='russian'))

    lowercase_tokens = [[token.lower() for token in m] for m in tokens]

    stop_words = set(stopwords.words('russian'))
    filtered_tokens = [[word for word in m if word not in stop_words] for m in lowercase_tokens]

    # print([word for word in m for m in filtered_tokens for word in m])
    return list(filter(lambda x: x, [re.sub(r'[^a-zA-Zа-яёА-ЯЁ@#\s]', '', word) 
                                     for m in filtered_tokens for word in m]))


tokenized_words = get_tokens(chat_df[~chat_df.text.str.startswith('File: <')])
print(*sorted(Counter(tokenized_words).items(),
              key=lambda x: x[1], reverse=True)[:50], sep='\n')
# filtered_tokens[:100]
# tokenized_words[:100]

In [ ]:
# Получение значимых слов для каждого участника по TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer


# Смотрим слова, которые встречаются чаще всего у пользователей чарез tf-idf
tfidf_data = (chat_df.groupby('from_id', as_index=False).text
                     .apply(lambda x: ' '.join(get_tokens(pd.DataFrame({'text': [' '.join(x)]})))))

# Векторизация текста с помощью TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tfidf_data.text)

# Получение значений TF-IDF в виде DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.index = tfidf_data.from_id

# print(tfidf_df)

# Функция для поиска слов с высоким TF-IDF
def get_high_tfidf_words(tfidf_row, threshold=0.00175):
    # print(tfidf_row[tfidf_row != 0.0])
    # print(tfidf_row > threshold)
    return tfidf_row[tfidf_row > threshold].index.tolist()

# Применение функции ко всем документам
# tfidf_data['valuable_words'] = tfidf_df.apply(get_high_tfidf_words, axis=1)
tfidf_df = (tfidf_df.apply(get_high_tfidf_words, axis=1)
                    .reset_index().rename(columns={0: 'words'})
                    .merge(members_df[
                        members_df.user_id.str.startswith('user') | members_df.user_id.str.startswith('channel')
                        ], 
                           how='left', left_on='from_id', right_on='user_id')
                    [['username', 'words']])

# Вывод результатов
for i, line in tfidf_df[(tfidf_df.words.apply(len) > 0) & ~tfidf_df.username.isna()].iterrows():
    # print(word_stat[word_stat.username == line.username].text.values[0][0])
    print(line.username, [w for w in word_stat[word_stat.username == line.username].text.values[0] 
                          if w[0] in line.words], end='\n\n')


In [ ]:
tfidf_words = []
for _, wlist in tfidf_df.iterrows():
    tfidf_words.extend(wlist.words)

tfidf_words = list(set(tfidf_words))
tfidf_words[:10]

In [ ]:
# ФОрмируем облако слов из токенов
from wordcloud import WordCloud


# is_tfidf = {t: t in tfidf_words for t in set(tokenized_words)}

# text_wordcloud = ' '.join([t for t in tokenized_words if is_tfidf[t]]) 
text_wordcloud = ' '.join(tokenized_words)
wordcloud = WordCloud(width=1200, height=600, max_font_size=200, max_words=100, 
                      colormap='Dark2_r', background_color="white").generate(text_wordcloud)

plt.figure(figsize=(14,7))
plt.imshow(wordcloud)
plt.title('Топ 100 слов чата МКН')
plt.axis("off")
plt.show()

In [ ]:
import pymorphy2


lemmatizer = pymorphy2.MorphAnalyzer()
lemmatized_tokens = [lemmatizer.parse(word)[0].normal_form for word in tokenized_words]
# lemmatized_tfidf = [lemmatizer.parse(word)[0].normal_form for word in tfidf_words]
print(*sorted(Counter(lemmatized_tokens).items(), 
              key=lambda x: x[1], reverse=True)[1:21], sep='\n')

In [ ]:
from wordcloud import WordCloud


# is_tfidf = {t: t in lemmatized_tfidf for t in set(lemmatized_tokens)}
# text_wordcloud = ' '.join([t for t in lemmatized_tokens if is_tfidf[t]]) 
text_wordcloud= ' '.join(lemmatized_tokens) 
wordcloud = WordCloud(width=1200, height=600, max_font_size=200, max_words=100, 
                      colormap='Dark2_r', background_color="white").generate(text_wordcloud)

plt.figure(figsize=(14,7))
plt.imshow(wordcloud)
plt.title('Топ 100 слов чата МКН (лемматизированные)')
plt.axis("off")
plt.show()

In [ ]:
# Персонализированное облако слов
uid = ''  
user_tokenized = get_tokens(chat_df[~chat_df.text.str.startswith('File: <') & (chat_df.from_id == uid)])

# is_tfidf = {t: t in tfidf_words for t in set(user_tokenized)}
# text_wordcloud = ' '.join([t for t in user_tokenized if is_tfidf[t]]) 

text_wordcloud= ' '.join(user_tokenized) 
wordcloud = WordCloud(width=1200, height=600, max_font_size=200, max_words=100, 
                      colormap='Dark2_r', background_color="white").generate(text_wordcloud)

plt.figure(figsize=(14,7))
plt.imshow(wordcloud)
plt.title('Топ 100 слов ' + members_df[members_df.user_id == uid].username.values[0])
plt.axis("off")
plt.show()

## Реакции

In [ ]:
reacts_df.emoji = reacts_df.emoji.fillna(reacts_df.document_id)
reacts_df

In [ ]:
members_reacts_df.date = pd.to_datetime(members_reacts_df.date, errors='coerce')
members_reacts_df.member_id = members_reacts_df.member_id.fillna('Unknown')
members_df = pd.concat([members_df, pd.DataFrame({'user_id': ['Unknown'], 'username': ['Unknown']})], axis=0, ignore_index=True)

In [ ]:
# Топ реакций

# print(memders_reacts_df.groupby('reaction_id', as_index=False).count()['message_id'].sum())
# print(memders_reacts_df.groupby('reaction_id', as_index=False).count()['member_id'].sum())
(members_reacts_df.groupby('reaction_id', as_index=False).count()
 [['reaction_id', 'message_id']]
                  .merge(reacts_df, how='left', left_on='reaction_id', right_on='id')
                  [['emoji', 'message_id']]
                  .sort_values('message_id', ascending=False)
                  .reset_index(drop=True)[:20])

In [ ]:
# Кто ставит больше всего реакций
reacts_count = (members_reacts_df.groupby('member_id', as_index=False).count()
                [['member_id', 'reaction_id']]
                .merge(members_df, how='left', left_on='member_id', right_on='user_id')
                [['username', 'reaction_id']]
                .sort_values('reaction_id', ascending=False)
                .reset_index(drop=True))
reacts_count[:20]

In [ ]:
# Соотношение реакций к количеству сообщений
reacts_ratio = reacts_count.merge(message_count, how='left', on='username')
reacts_ratio.reaction_id = reacts_ratio.reaction_id / reacts_ratio.id
reacts_ratio.sort_values('reaction_id', ascending=False).reset_index(drop=True)[['username', 'reaction_id']][:20]

In [ ]:
mess_reacts = (members_reacts_df.rename(columns={'date': 'rdate'})
                                .merge(chat_df, how='left', left_on='message_id', right_on='id'))
mess_reacts = mess_reacts[['message_id', 'from_id', 'date', 'reaction_id', 'member_id', 'rdate']]
mess_reacts

In [ ]:
# Кому больше всего ставят реакций
(mess_reacts.groupby('from_id', as_index=False).count()
            .sort_values('reaction_id', ascending=False)
            .reset_index(drop=True)
            .merge(members_df, left_on='from_id', right_on='user_id')
            [['username', 'reaction_id']][:20])

In [ ]:
# Кто кому больше всего ставит реакций
(mess_reacts.groupby(['from_id', 'member_id'], as_index=False).count()
            .sort_values('reaction_id', ascending=False)
            .reset_index(drop=True)
            .merge(members_df, how='left', left_on='member_id', right_on='user_id')
            [['from_id', 'username', 'reaction_id']]
            .rename(columns={'username': 'ruser'})
            .merge(members_df, how='left', left_on='from_id', right_on='user_id')
            [['ruser', 'username', 'reaction_id']][:20])

In [ ]:
# Сколько уникальных аккаунтов отреагировали на все сообщения участника чата
(mess_reacts.groupby('from_id', as_index=False).member_id.nunique()
            .sort_values('member_id', ascending=False)
            .reset_index(drop=True)
            .merge(members_df, how='inner', left_on='from_id', right_on='user_id')
            [['username', 'member_id']][:20])

In [ ]:
# Скольким уникальным аккаунтам участники чата оставили реакции
(mess_reacts.groupby('member_id', as_index=False).from_id.nunique()
            .sort_values('from_id', ascending=False)
            .reset_index(drop=True)
            .merge(members_df, how='inner', left_on='member_id', right_on='user_id')
            [['username', 'from_id']][:20])

In [ ]:
# Сообщения с самым большим количеством реакций (от пользователей)
(mess_reacts.groupby('message_id', as_index=False).count()
            # .sort_values('reaction_id', ascending=False).reset_index(drop=True)
            [['message_id', 'reaction_id']]
            .merge(chat_df, how='left', left_on='message_id', right_on='id')
            # [['message_id', 'from_id', 'text', 'reaction_id', 'date']]
            .merge(members_df[~members_df.user_id.str.startswith('channel')], 
                   how='inner', left_on='from_id', right_on='user_id')
            [['message_id', 'username', 'text', 'reaction_id', 'date']]
            .sort_values(['reaction_id', 'message_id'], ascending=[False, True])
            .reset_index(drop=True)[:20])

In [ ]:
# Сообщения с самым большим количеством реакций (от каналов)
(mess_reacts.groupby('message_id', as_index=False).count()
            # .sort_values('reaction_id', ascending=False).reset_index(drop=True)
            [['message_id', 'reaction_id']]
            .merge(chat_df, how='left', left_on='message_id', right_on='id')
            # [['message_id', 'from_id', 'text', 'reaction_id', 'date']]
            .merge(members_df[members_df.user_id.str.startswith('channel')], 
                   how='inner', left_on='from_id', right_on='user_id')
            [['message_id', 'username', 'text', 'reaction_id', 'date']]
            .sort_values(['reaction_id', 'message_id'], ascending=[False, True])
            .reset_index(drop=True)[:20])

In [ ]:
# Кто каких реакций больше всего получает
(mess_reacts[~mess_reacts.reaction_id.isin([1, 3])].groupby('from_id', as_index=False).reaction_id.value_counts()
            [['from_id', 'reaction_id', 'count']]
            .sort_values(['count', 'from_id', 'reaction_id'], ascending=[False, True, True])
            .groupby('from_id', as_index=False).agg({'count': 'max', 'reaction_id': 'first'})
            .merge(members_df, how='left', left_on='from_id', right_on='user_id')
            .merge(reacts_df, how='left', left_on='reaction_id', right_on='id')
            [['username', 'emoji', 'count']]
            .sort_values(['count', 'username'], ascending=[False, True])
            .reset_index(drop=True)[:20])

In [ ]:
# Кто какие реакции больше всего ставит
(mess_reacts[~mess_reacts.reaction_id.isin([1, 3])].groupby('member_id', as_index=False).reaction_id.value_counts()
            .sort_values(['count', 'member_id', 'reaction_id'], ascending=[False, True, True])
            .groupby('member_id', as_index=False).agg({'count': 'max', 'reaction_id': 'first'})
            .merge(members_df, how='left', left_on='member_id', right_on='user_id')
            .merge(reacts_df, how='left', left_on='reaction_id', right_on='id')
            [['username', 'emoji', 'count']]
            .sort_values(['count', 'username'], ascending=[False, True])
            .reset_index(drop=True)[:20])

In [ ]:
# Какие реакции кто ставит
reacts_count = (mess_reacts.merge(members_df, how='left', left_on='member_id', right_on='user_id')
                           .groupby('reaction_id', as_index=False).username.apply(list))
reacts_count.username = reacts_count.username.apply(lambda x: list(
    sorted([(m, c) for m, c in Counter(x).items()], key=lambda y: -y[1])))
reacts_count = reacts_count.merge(reacts_df, left_on='reaction_id', right_on='id')[['emoji', 'username']]

for react, un in sorted(zip(reacts_count.emoji, reacts_count.username), 
                        key=lambda x: sum(k for _, k in x[1]), reverse=True):
    print(react, un, sep='    ', end='\n\n')


In [ ]:
# Кому от кого и сколько каких реакций получил
members_reacts_count = (mess_reacts.merge(members_df, how='left', left_on='member_id', right_on='user_id')
                                   .groupby(['from_id', 'reaction_id'], as_index=False).username
                                   .apply(list)
                                   .rename(columns={'username': 'ruser'})
                                   .merge(reacts_df, left_on='reaction_id', right_on='id')
                                   .merge(members_df, how='left', left_on='from_id', right_on='user_id')
                                   [['username', 'emoji', 'ruser']])

members_reacts_count.ruser = members_reacts_count.ruser.apply(
    lambda x: list(sorted([(m, c) for m, c in Counter(x).items()], key=lambda y: -y[1])))
members_reacts_count.emoji = members_reacts_count.apply(lambda x: (x.emoji, x.ruser), axis=1)
members_reacts_count = members_reacts_count.groupby('username', as_index=False).emoji.apply(
    lambda x: sorted(list(x), key=lambda y: sum(k for _, k in y[1]), reverse=True))

for un, react in sorted(zip(members_reacts_count.username, members_reacts_count.emoji), 
                        key=lambda x: sum(k for _, y in x[1] for _, k in y), reverse=True):
    print(un, end='\n\n')
    print(*react, sep='\n\n', end='\n\n\n')


In [ ]:
react_time = mess_reacts[:]
react_time.rdate = mess_reacts.rdate - mess_reacts.date
react_time = react_time.sort_values('rdate').reset_index(drop=True).dropna()
react_time

In [ ]:
# График распределения времени реакций
(react_time.rdate.groupby(react_time[react_time.rdate.dt.seconds <= 60].rdate.dt.floor('1s')).count()
           .plot.area(figsize=(14, 6), title='Количество реакций, которые были поставлены спустя указанное время'))

In [ ]:
uid = ''  # идентификатор пользователя для индивидуального графика распределения времени реакций
# [react_time.rdate.dt.days <= 30]
(react_time[react_time.member_id == uid].rdate.groupby(react_time[react_time.rdate.dt.seconds <= 60].rdate.dt.floor('1s')).count()
           .plot.area(figsize=(14, 6), title='Количество реакций, которые поставил участник чата спустя указанное время\n' 
                      f'{members_df.loc[members_df.user_id == uid, ["username"]].values[0][0]}'))